# SDSS Tutorial #5:  Modeling Star Clusters with H-R Diagrams

[Prof. Britt Lundgren, University of North Carolina Asheville](https://astrophysics.wp.unca.edu/britt-lundgren/)


## Objective
 - Using Sloan Digital Sky Survey (SDSS) data to create an H-R diagram for a star cluster, and model its age, metallicity, and distance.
 
## Learning Outcomes

By the end of this notebook, you should be able to: 
 - Query photometric data for a star cluster in the SDSS
 - Visualize the cluster data in an H-R Diagram
 - Apply model isochrones to the data to estimate the cluster's properties and distance.
 

## Prerequisites

* This notebook assumes that the user has basic proficiency in the **Python** programming language.
* This notebook uses **SQL queries** to select data from the SDSS database. If the SQL language and/or the SDSS database is new to you, we recommend this [brief tutorial](http://cas.sdss.org/dr16/en/help/howto/search/searchhowtohome.aspx). 
* This notebook is intended to be run in the [SciServer Compute](http://www.sciserver.org/tools/compute/) platform. If you have not used SciServer before, please review the [first notebook](https://escip.io/notebooks/sdss_tutorial_1.html) in this series of activities. 

Before we begin, let's fetch some required files:

In [2]:
# Import Python libraries to work with SciServer
import SciServer.CasJobs as CasJobs # query with CasJobs
import SciServer.SkyServer as SkyServer # look up objects and generate images
print('SciServer libraries imported')

# Import some other needed libraries
import pandas
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
print('Other libraries imported')

# Apply some settings

# ensure columns get written completely in notebook
pandas.set_option('display.max_colwidth', -1)
# do *not* show python warnings 
import warnings
warnings.filterwarnings('ignore')
print('Settings applied')

# Import required images for this notebook
import os

if not os.path.exists('isochrones'):
    !mkdir isochrones
    
imgs = ['tmp1487212885.iso','tmp1553190074.iso', 'tmp1553191176.iso']
for i in imgs:
    if not os.path.exists('img/'+i):
        os.system('wget --no-check-certificate --content-disposition https://github.com/brittlundgren/ASTR420/raw/update_2023/isochrones/'+i)
        os.system('mv %s isochrones/' % i)
        
print('files imported')

SciServer libraries imported
Other libraries imported
Settings applied
files imported


**NOTE: In order for embedded images to display properly in this notebook, it may be necessary to restart the kernel and reopen this notebook before continuing.**

## Introduction

In this tutorial we will apply numerical models of stellar evolution to determine the distance, metallicity, and age of globular clusters observed in the SDSS. You may find it useful to refer to the project background and helpful hints posted here: http://voyages.sdss.org/hr-diagrams/

## Choosing a star cluster

The first, and very important, step is selecting a cluster with good photometric data. If you already have a favorite cluster, type it into the “name” box in [Navigate](http://skyserver.sdss.org/dr17/en/tools/chart/navi.aspx) and click “Resolve” to see if it’s in the SDSS database. If not, you can look at [this catalog](http://www.galaxyzooforum.org/index.php?topic=278836.0) of open and globular clusters that have already been found with SDSS data and choose one that you like. When you’ve found an object, type its name into Navigate as just explained, and it will likely appear automatically. Some of the clusters aren’t listed in the SDSS index, however; if that’s the case, each object’s RA and Dec can be found by clicking on its image in the catalog. You can then type the values into Navigate by hand.

While selecting a cluster, don’t be afraid to look closely at several in order to compare your options and find the one best suited to your needs—not every cluster in the database will produce a good HR diagram. Ask yourself:

- What kinds of data do you need to build a Hertzsprung-Russell diagram?
- How many data points (stars) will be necessary to see a clear and distinct trend?
- Why do some clusters have better photometry than others? How will that affect your plot?
- What are the advantages/disadvantages to changing the radius of your search?

## Fetch the data for the star cluster of choice

You can find a resource to get you started with your own SQL searches [here](http://voyages.sdss.org/sql-for-cmds/).

To start your query, think about exactly which data you’re looking for and what criteria you’ll use to select it. You can explore the [Schema Browser](http://skyserver.sdss.org/dr14/en/help/browser/browser.aspx) to better understand the all of the information at your disposal in the SDSS database.  While answering the questions below you will likely want to review the data model for the “PhotoObj” schema, which provides from the best photometric data available. In general, it may be a good idea to request more options than you think you’ll need, in case you change your mind later.

Several magnitude-fitting models are available in PhotoObj: psfMag, fiberMag, petroMag, deVMag, expMag, modelMag, cModelMag, and dered.   You can find them defined [here](http://skyserver.sdss.org/dr14/en/help/browser/browser.aspx#&&history=description+PhotoPrimary+V).
- How do they differ? 
- Which is best suited to the plot you’ll be making, where brightness and color are the two important variables?


In [3]:
## Create a query to fetch stars for one of the following globular clusters
# Pal 3: RA = 151.3801, Dec = +0.072
# Pal 5: RA = 229.0128, Dec = -0.1082 

# Enter the coordinates of your star cluster (to be used to construct the query)
cluster_name = 'Palomar 5'
cluster_ra = 229.0128  # RA of the center of the cluster (in decimal degrees)
cluster_dec = -0.182   # Dec of the center of the cluster (in decimal degrees)
cluster_radius = 4.0   # Radius of the cluster, in arcminutes
cluster_distance = 23000 # approximate distance to your cluster, in units of parsecs

query = 'SELECT p.objid, p.dered_u AS u, p.psfMagErr_u as u_err, p.dered_g AS g, p.psfMagErr_g as g_err, ' # de-reddened mags
query += 'p.dered_r AS r, p.psfMagErr_r as r_err, p.dered_i AS i, p.psfMagErr_i as i_err, '
query += 'p.dered_z AS z, p.psfMagErr_z as z_err '
query += 'FROM photoObj p '
query += 'JOIN dbo.fGetNearbyObjEq(' + str(cluster_ra) + ', ' + str(cluster_dec) + ', ' + str(cluster_radius) + ') n '
query += 'ON p.objid = n.objID '
query += 'WHERE p.type = 6 '
query += 'AND p.psfMag_g BETWEEN -10 AND 23 '
query += 'AND p.psfMag_u/p.psfMagerr_u > 30 '
query += 'AND p.psfMag_g/p.psfMagerr_g > 30 '
query += 'AND p.psfMag_r/p.psfMagerr_r > 30 '
query += 'AND p.psfMag_i/p.psfMagerr_i > 30 '
query += 'AND p.psfMag_z/p.psfMagerr_z > 30'
#print(query)   # useful for debugging, remove first # to uncomment

# send query to CasJobs
stars = CasJobs.executeQuery(query, "dr16")

stars

,objid,u,u_err,g,g_err,r,r_err,i,i_err,z,z_err
0,1237648721252188401,20.56440,0.053427,18.25084,0.014385,17.22990,0.012818,16.85635,0.012288,16.65042,0.022054
1,1237648721252190019,23.21074,0.555454,21.90948,0.102005,21.37207,0.076211,21.42161,0.114987,21.32529,0.342055
2,1237648721252188402,20.76552,0.063972,19.58318,0.025416,19.07266,0.021199,18.87590,0.021882,18.77323,0.045118
3,1237648721252188411,22.26196,0.181122,21.55358,0.053783,21.30930,0.064437,21.34820,0.101640,21.60409,0.465597
4,1237648721252188404,21.67508,0.116361,20.33988,0.023511,19.98136,0.023396,19.91230,0.028822,19.87088,0.095213
5,1237648721252188399,17.99146,0.010977,16.63934,0.013478,16.14954,0.012485,15.95861,0.011994,15.86932,0.021170
6,1237648721252189051,19.76357,0.028388,18.29830,0.014359,17.72178,0.013107,17.48829,0.012722,17.33437,0.023975
7,1237648721252189019,23.98232,0.624054,21.77448,0.120199,21.44852,0.122616,21.47629,0.165443,20.82882,0.204787
8,1237648721252189017,22.00825,0.186254,20.99001,0.053943,20.68993,0.052725,20.71252,0.068448,20.73690,0.206071
9,1237648721252188406,21.89254,0.134773,20.89958,0.032271,20.65372,0.035135,20.56679,0.044999,20.66575,0.192075


## Ploting an H-R Diagram (magnitude vs. color)

Of the five ugriz filters, you will need to use two in your plot. Which ones will you use? Refer to the “Color” section of the SDSS Voyages [Star Color](http://voyages.sdss.org/star-color/) site.

Our end goal is to produce a Hertzprung-Russell (H-R) diagram, so make sure your axes are going in the right direction. You can check this intuitively by considering what the axes actually represent: luminosity is how much energy an object emits, so when luminosity increases, how should the apparent magnitude be affected? And since an HR diagram plots decreasing temperature on the x-axis, what does that imply about the color of light being emitted? Should it go from bluer stars to redder stars, or the other way around?

In [4]:
# YOUR CODE HERE

## Making sense of your plot

A successful color-magnitude diagram can take many shapes. How many of the following characteristic features of an H-R diagram can you locate?
    - Main Sequence
    - H-core exhaustion (MS turn-off point)
    - Red Giant Branch
    - Horizontal Branch
    - Asymptotic Giant Branch
    
Your cluster may look very different from a typical H-R diagram, especially since the SDSS telescope is limited in its range from about 14th to 24th magnitude. This means that you’re looking at a horizontal cross-section of the full cluster, so don’t be discouraged if all of its features aren’t visible.

Try re-plotting your H-R Diagram with any identifiable features labeled. The following links demonstrate how to add text annotations and arrow to your plot:
- http://matplotlib.org/users/text_intro.html
- http://matplotlib.org/examples/pylab_examples/arrow_simple_demo.html

In [5]:
# YOUR CODE HERE

## Modeling stellar evolution tracks with isochrones

An isochrone is a computational model of stellar evolution. Parameters for a cluster like its age and chemical composition are plugged into a formula, and a computer simulates the development of stars within that cluster based upon their mass. The output is similar to the results of your SQL query from the first section of this Expedition: it’s a table of numbers, where each row lists the properties of a given star. That means you can also plot this data just like you did to produce your color-magnitude diagram, and it should look like an HR diagram for a cluster of the properties you selected.

Isochrones are designed to model the internal mechanisms of stars, so we use them to test our understanding of stellar evolution by comparing their results against external (measurable) properties. This is why we don’t plot them on the mass-luminosity axes or using other relationships: we can most easily measure stars’ luminosity and temperature, which makes Hertzsprung-Russell diagrams the most efficient format to place theory alongside observation. We’ll be doing things backwards, though, since we assume that isochrones are more or less correct and use them to learn more about your cluster.

When producing an isochrone to match your cluster, the primary variables you need to work with are the metallicity and age. These are two parameters that every isochrone generator should require, although they may offer additional options as well. 

Metallicity is often expressed as an abundance ratio, defined as the logarithm of the ratio of a star's iron abundance compared to that of the Sun. It is defined as

$[Fe/H]=\log_{10}{\left({\frac {N_{Fe}}{N_{H}}}\right)_{\text{star}}}-\log _{10}{\left({\frac {N_{\text{Fe}}}{N_{\text{H}}}}\right)_{\text{sun}}}$

where $N_{Fe}$ and $N_{H}$ are the number of iron and hydrogen atoms per unit of volume respectively. By this formulation, stars with a higher metallicity than the Sun have a positive logarithmic value, whereas those with a lower metallicity than the Sun have a negative value. For example, stars with a [Fe/H] value of +1 have 10 times the metallicity of the Sun; conversely, those with a [Fe/H] value of −1 have 1/10, while those with a [Fe/H] value of 0 have the same metallicity as the Sun, and so on. 

To simplify the process of guessing which metallicity to use in generating your model isochrone, feel free to do an online search for your cluster’s metallicity. The result should be a value for Z or [Fe/H], and depending on which one your generator uses, you may need to convert these values. The equation for this is [Fe/H] = log(Z/0.02), using a base-10 logarithm. 

As for the age, you’ll be finding that for your cluster in the next section of the activity. For now, plug in 200 million (2•10⁸) years; this should offer a relatively long main sequence to compare against.

A brief online search can bring up several results for isochrone generators, typically hosted on university web domains, so feel free to explore your options before choosing which one to use. We will use the [Dartmouth Isochrone and Luminosity Function Generator 2012](http://stellar.dartmouth.edu/models/index.html). It’s important to note, however, that models of stellar evolution are constantly being developed as we deepen our knowledge of the processes involved and our computing power improves. The generators linked to above may soon obsolesce, so you may want to research if other options have been released recently.

Using the Dartmouth Generator, create an isochrome model with a combination of the following choices: 
- [Fe/H] options: [-2.5, -2.0, -1.5, -1.0, -0.5, 0.0, +0.2, +0.3, +0.5]
- [alpha/H] options: [-0.2, 0.0, +0.2, +0.4, +0.6, +0.8]


If you have chosen a globular cluster, note that their very old stellar populations mean that the stars have extremely low metallicities (as they formed when the universe was young and metal-poor). 

Make sure you also select SDSS filters for your output!

Produce an isochrome using the Dartmouth web generator, and upload the *.iso file to your SciServer dashboard.



In [8]:
## Upload the isochrone file you have created into the "isochrones" folder, and
## EDIT the name and location below to point to your isochrome model file:

isofile = "isochrones/tmp1487212885.iso"  # [Fe/H]=-2.0
print('Isochrone file loaded: '+isofile)

Isochrone file loaded: isochrones/tmp1487212885.iso


In [18]:
## Prepared for you below is a function that will 
## read in the models from your uploaded isochrome file.
def read_isofile(filename):

    # Set a minimum and maximum age range (in Gyrs) below:
    minage = 11.0
    maxage = 12.0
    sdss_u = []
    sdss_g = []
    sdss_r = []
    sdss_i = []
    sdss_z = []

    age = [] # in Gigayears
    mass = [] # ratio of stellar mass to M_sun
    logT = [] # log of the effective Temperature (K)
    logL = [] # log of the ratio of stellar luminosity to L_sun
    counter=0
    for line in open(filename).readlines():
        counter+=1
        cols = line.split()
        if (line.startswith('#')):
    #        print(counter, ': starts with hashtag: ',cols)
            if (counter >= 8):   # real data starts on line #8
                if ("AGE" in cols[0]):
                    block = (line[0:11])
                    thisage = float(block.split('=')[1])
    #                print("Reading stellar models with age: {0:.2f} Gyr".format(thisage))            
        else:
            if (thisage > minage) and (thisage < maxage):
#                print(cols)
                if (len(cols) > 0):
                    age.append(thisage)
                    mass.append(float(cols[1]))
                    logT.append(float(cols[2]))
                    logL.append(float(cols[4]))
                    sdss_u.append(float(cols[5]))
                    sdss_g.append(float(cols[6]))
                    sdss_r.append(float(cols[7]))
                    sdss_i.append(float(cols[8]))
                    sdss_z.append(float(cols[9]))
    return (sdss_u, sdss_g, sdss_r, sdss_i, sdss_z, age, mass, logT, logL)    

(sdss_u, sdss_g, sdss_r, sdss_i, sdss_z, age, mass, logT, logL) = read_isofile(isofile)
#(sdss_u2, sdss_g2, sdss_r2, sdss_i2, sdss_z2, age2, mass2, logT2, logL2) = read_isofile(isofile2)
#(sdss_u3, sdss_g3, sdss_r3, sdss_i3, sdss_z3, age3, mass3, logT3, logL3) = read_isofile(isofile3)

print("Read in {0:.0f} stellar models, ranging in age from {1:.2f} to {2:.2f} Gyr.".format(len(sdss_u), age[0], age[-1]))

Read in 267 stellar models, ranging in age from 11.50 to 11.50 Gyr.


## Now let's overplot the model on the H-R diagram

In [9]:
# YOUR CODE HERE


## Distance, extinction and reddening

That probably looks like an awfully bad fit. That is because our data are in apparent, reddened magnitudes and the isochrones provide absolute, unreddened magnitudes.

Now might be a good time to go back and ensure that you are selecting de-reddened ("dered") magnitudes in your SDSS star cluster query.  If you weren't, re-run your query and the steps that followed.

With that done, we'll also now try using our model to calculate the distance to the cluster.  To do this, we want to apply the [distance modulus](https://en.wikipedia.org/wiki/Distance_modulus), which relates apparent and absolute magnitudes with distance:

$m-M=5\log _{10}(d)-5=5\log _{10}\left({\frac {d}{10\,\mathrm {pc} }}\right)$

where $m$ is the apparent magnitude, $M$ is the absolute magnitude, and $d$ is the distance to the cluster in units of parsecs.

Try guessing a distance to your cluster, in units of parsecs, and compute the distance modulus, using the calculation below. Then correct your model magnitudes using the distance modulus equation.  By guessing and overplotting a number of times, you will hopefully find the position where the model and data overlap.  Such agreement indicates that you have approximately found the distance to the cluster!

In [10]:
# YOUR CODE HERE

## Now let's plot again, to check whether our model and data are in agreement for the distance to the cluster

If the isochrone is sitting to the left of the data, this may be due to interstellar dust extinction, which both absorbs light and reddens colours. The relationship between reddening  E(B−V) and extinction  AV  is:

AV=3.2*E(B−V)

(e.g., For NGC 7789, E(B−V) is roughly 0.2)

Try shifting your model to correct for the extinction, if necessary.

In [11]:
# Adust the value of "ebv", representing E(B-V), below:
ebv = 0.2

av = 3.2*ebv

model_g_dered = np.array(model_g_corr) + av
model_gr_dered = np.array(model_color) + ebv
print('Done')

NameError: name 'model_g_corr' is not defined

In [1]:
# YOUR CODE HERE

## Determining the age of the cluster

Any additional offset between your model and the data is likely due to the cluster age.  Try going back to the earlier step where you created your model, and change the age range that's plotted.  With some experimentation, can you fit the age of the cluster, simultaneously with its distance and reddening?  

In [12]:
# YOUR CODE HERE